In [186]:
############################################################
# CIS 521: Homework 4
############################################################

student_name = "Type your full name here."

############################################################
# Imports
############################################################

# Include your imports here, if any are used.

import collections
import copy
import itertools
import random
import math
import numpy as np

############################################################
# Section 1: Dominoes Game
############################################################


def create_dominoes_game(rows, cols):
    board = [[False for i in range(cols)] for j in range(rows)]
    return DominoesGame(board)


class DominoesGame(object):

    # Required
    def __init__(self, board):
        self.board = board
        self.board_rows = len(board)
        self.board_cols = len(board[0])

    def get_board(self):
        return self.board

    def reset(self):
        for i in range(self.board_rows):
            for j in range(self.board_cols):
                self.board[i][j] = False

    def is_legal_move(self, row, col, vertical):
        # check if outside boundary
        if row < 0 or row >= self.board_rows or col < 0 or col >= self.board_cols:
            return False
   
        if vertical:
            if row + 1 >= self.board_rows:
                return False
            if self.board[row][col] or self.board[row + 1][col]:
                return False
        else:
            if col + 1 >= self.board_cols:
                return False
            if self.board[row][col] or self.board[row][col + 1]:
                return False
        return True
    
    def legal_moves(self, vertical):
        for i in range(self.board_rows):
            for j in range(self.board_cols):
                if self.is_legal_move(i, j, vertical):
                    yield (i, j)

    def perform_move(self, row, col, vertical):
        self.board[row][col] = True
        if vertical:
            self.board[row + 1][col] = True
        else:
            self.board[row][col + 1] = True
        

    def game_over(self, vertical):
        for i in range(self.board_rows):
            for j in range(self.board_cols): 
                if self.is_legal_move(i,j,vertical):
                    return False
        return True

    def copy(self):
        copy_board = copy.deepcopy(self.board)
        return DominoesGame(copy_board)

    def successors(self, vertical):
        legal_moves = self.legal_moves(vertical)
        for move in legal_moves:
            copy = self.copy()
            copy.perform_move(move[0], move[1], vertical)
            yield move, copy

    def get_random_move(self, vertical):
        legal_moves = self.legal_moves(vertical)
        return random.choice(legal_moves)

    # Required
    
    def get_best_move(self, vertical, limit):
        if vertical:
            move, value, total_leaves = self.alpha_beta_max(0, limit, -np.inf, np.inf)
        else:
            move, value, total_leaves = self.alpha_beta_min(0, limit, -np.inf, np.inf)
            value = value * -1
        return ((move[0], move[1]), value, total_leaves)
    
    # Create two methods for max and min
    def alpha_beta_max(self, vertical, limit, alpha, beta):
        # check termination condition
        if self.game_over(True) or limit == vertical:
            # calculate best value as num_player_moves(original vertical) - num_opp_moves(~original_vertical)
            # return best_move, best_val, total_leaves
            return None, len(list(self.legal_moves(True))) - len(list(self.legal_moves(False))), 1
        
        #initialze return variables
        best_move = None
        value = -np.inf
        total_leaves = 0
    
        for move, new_game in self.successors(True): # try to find max of self's successors
            new_move, new_value, new_leaves = new_game.alpha_beta_min(vertical+1, limit, alpha, beta)
            # update total_leaves
            total_leaves += new_leaves
            # update variabels if new max found
            if new_value > value:
                value = new_value
                best_move = move
            if new_value >= beta:
                # break if the prining check is true
                break
            if new_value > alpha:
                # update alpha
                alpha = new_value
        return best_move, value, total_leaves
    
    def alpha_beta_min(self, vertical, limit, alpha, beta):
        # similar to alpha_beta_max
        # check termination condition
        if self.game_over(False) or limit == vertical:
            # calculate best value as num_player_moves(original vertical) - num_opp_moves(~original_vertical)
            # return best_move, best_val, total_leaves
            return None, len(list(self.legal_moves(True))) - len(list(self.legal_moves(False))), 1
            
        # initialze return variables
        best_move = None
        value = np.inf
        total_leaves = 0
        
        for move, new_game in self.successors(False): # try to find max of self's successors
            new_move, new_value, new_leaves = new_game.alpha_beta_max(vertical+1, limit, alpha, beta)
            # update total_leaves
            total_leaves += new_leaves
            # update variabels if new max found
            if new_value < value:
                value = new_value
                best_move = move
            if new_value <= alpha:
                # break if the prining check is true
                break
            # break if the prining check is true
            if new_value < beta:
                # update beta
                beta = new_value
        return best_move, value, total_leaves
    

############################################################
# Section 2: Feedback
############################################################

feedback_question_1 = """
Type your response here.
Your response may span multiple lines.
Do not include these instructions in your response.
"""

feedback_question_2 = """
Type your response here.
Your response may span multiple lines.
Do not include these instructions in your response.
"""

feedback_question_3 = """
Type your response here.
Your response may span multiple lines.

Do not include these instructions in your response.
"""


In [187]:
g = create_dominoes_game(3,3)
g.get_board()

[[False, False, False], [False, False, False], [False, False, False]]

In [188]:
a = [[True, False], [True, False]]
g = DominoesGame(a)
g.reset()
g.get_board()

[[False, False], [False, False]]

In [189]:
b = [[False, False], [False, False]]
g = DominoesGame(b)
print(g.is_legal_move(0, 0, True))
print(g.is_legal_move(0, 0, False))

True
True


In [190]:
b = [[True, False], [True, False]]
g = DominoesGame(b)

print(g.is_legal_move(0, 0, False) == False)
print(g.is_legal_move(0, 1, True) == True)
print(g.is_legal_move(1, 1, True) == False)

True
True
True


In [191]:
g = create_dominoes_game(3, 3)
print(list(g.legal_moves(True)) == [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)])
print(list(g.legal_moves(False))==[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)])
print(len(list(g.legal_moves(False))))
g.get_board()

True
True
6


[[False, False, False], [False, False, False], [False, False, False]]

In [192]:
b = [[True, False], [True, False]]
g = DominoesGame(b) 
print(list(g.legal_moves(True)) == [(0, 1)])
print(list(g.legal_moves(False)) == [])

True
True


In [193]:
g = create_dominoes_game(3, 3)
g.perform_move(0, 1, True)
print(g.get_board() == [[False, True,  False],
  [False, True,  False],
  [False, False, False]])


True


In [194]:
g = create_dominoes_game(3, 3)
g.perform_move(1, 0, False)
print(g.get_board() == [[False, False, False],
  [True,  True,  False],
  [False, False, False]])

True


In [195]:
g = create_dominoes_game(4, 4)
g2 = g.copy()
print(g.get_board() == g2.get_board())

True


In [196]:
g = create_dominoes_game(4, 4)
g2 = g.copy()
g.perform_move(0, 0, True)
(g.get_board() == g2.get_board()) == False

True

In [197]:
b = [[False, False], [False, False]]
g = DominoesGame(b)
for m, new_g in g.successors(True):
  print(m, new_g.get_board())
#  (0, 0) [[True, False], [True, False]]
#  (0, 1) [[False, True], [False, True]]

(0, 0) [[True, False], [True, False]]
(0, 1) [[False, True], [False, True]]


In [198]:
b = [[True, False], [True, False]]
g = DominoesGame(b)
# g.evaluate(True) # 1
# g.evaluate(False) # - 1
print(len(list(g.legal_moves(True))))


1


In [199]:
b = [[True, False], [True, False]]
g = DominoesGame(b)
for m, new_g in g.successors(True):
  print(m, new_g.get_board())
#(0, 1) [[True, True], [True, True]]

(0, 1) [[True, True], [True, True]]


In [200]:
b = [[False] * 3 for i in range(3)]
g = DominoesGame(b)
g.get_best_move(True, 1)
print(g.get_best_move(True, 1))
#((0, 1), 2, 6)
g.get_best_move(True, 2)
#((0, 1), 3, 10)

((0, 1), 2, 6)


((0, 1), 3, 10)

In [201]:
b = [[False] * 3 for i in range(3)]
g = DominoesGame(b)
g.perform_move(0, 1, True)
print(g.get_best_move(False, 1))
#((2, 0), -3, 2)
print(g.get_best_move(False, 2))
#((2, 0), -2, 5)

((2, 0), -3, 2)
((2, 0), -2, 5)
